In [0]:
import pandas as pd
# import sqlite3
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from wordcloud import WordCloud
import re
import os
from sqlalchemy import create_engine # database connection
import datetime as dt
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem.snowball import SnowballStemmer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.multiclass import OneVsRestClassifier
from sklearn.linear_model import SGDClassifier
from sklearn import metrics
from sklearn.metrics import f1_score,precision_score,recall_score
from sklearn import svm
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from datetime import datetime
from bs4 import BeautifulSoup
from sklearn.model_selection import GridSearchCV

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [0]:
import pandas as pd
import numpy as np
import pickle
from keras.preprocessing.text import Tokenizer
from keras.models import Sequential,load_model
from keras.layers import Activation, Dense, Dropout
from sklearn.preprocessing import LabelBinarizer
import sklearn.datasets as skds
from pathlib import Path
from sklearn.model_selection import train_test_split


from __future__ import print_function

from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from keras.layers import Embedding
from keras.layers import Conv1D, GlobalMaxPooling1D

from keras import layers

import json

Using TensorFlow backend.


In [0]:
df=pd.read_csv('./blurb_ml.csv')
nRow, nCol = df.shape
print(f'There are {nRow} rows and {nCol} columns')

There are 77109 rows and 4 columns


In [0]:
df.Genres=df.Genres.apply(lambda x: json.loads(x))
df

,Unnamed: 0,Name,Genres,Summary
0,0,Teenage Mutant Ninja Turtles: The Box Set Volu...,"[Graphic Novels and Manga, Fiction]",TMNT co-creator Kevin Eastman and writer Tom W...
1,1,Betty and Veronica: Fairy Tales,"[Children’s Middle Grade Books, Children’s Books]",Take a magical trip down Storybook Lane with t...
2,2,Friends and Fauxs,"[Women’s Fiction, Fiction]","Tracie Howard is back with all of the Gucci, g..."
3,3,Anti-Education,"[Philosophy, Religion and Philosophy, Nonfiction]","AN NYRB Classics OriginalIn 1869, at the age o..."
4,4,Fallout,"[Espionage Mysteries, Mystery and Suspense, Fi...",A first-class letter–containing a single sheet...
...,...,...,...,...
77104,77104,Design and Crime (And Other Diatribes),"[Art, Politics, Nonfiction]",In these diatribes on the marketing of culture...
77105,77105,The Things We Cherished,"[Historical Romance, Literary Fiction, Fiction]",Charlotte Gold is shocked when her ex-fiancé B...
77106,77106,"National Geographic Traveler: Madrid, 2nd Edition","[Travel: Europe, Travel, Nonfiction]",All the travel experts agreeconsumers want mo...
77107,77107,Hate Crime,[Nonfiction],"On June 7, 1998, James Byrd Jr., a forty-nine-..."


In [0]:
import nltk

In [0]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [0]:
df = df.reset_index(drop=True)
REPLACE_BY_SPACE_RE = re.compile('[/(){}\[\]\|@,;]')
BAD_SYMBOLS_RE = re.compile('[^0-9a-z #+_]')
STOPWORDS = set(stopwords.words('english'))

def clean_text(text):
    """
        text: a string
        
        return: modified initial string
    """
    text = text.lower() # lowercase text
    text = REPLACE_BY_SPACE_RE.sub(' ', text) # replace REPLACE_BY_SPACE_RE symbols by space in text. substitute the matched string in REPLACE_BY_SPACE_RE with space.
    text = BAD_SYMBOLS_RE.sub('', text) # remove symbols which are in BAD_SYMBOLS_RE from text. substitute the matched string in BAD_SYMBOLS_RE with nothing. 
    text = text.replace('x', '')
#    text = re.sub(r'\W+', '', text)
    text = ' '.join(word for word in text.split() if word not in STOPWORDS) # remove stopwors from text
    return text

In [0]:
df['Summary'] = df['Summary'].apply(clean_text)

In [0]:
def gdd(x):
    a=x.split()
    return len(a)

In [0]:

def decontracted(phrase):
    # specific
    phrase = re.sub(r"won't", "will not", phrase)
    phrase = re.sub(r"can\'t", "can not", phrase)

    # general
    phrase = re.sub(r"n\'t", " not", phrase)
    phrase = re.sub(r"\'re", " are", phrase)
    phrase = re.sub(r"\'s", " is", phrase)
    phrase = re.sub(r"\'d", " would", phrase)
    phrase = re.sub(r"\'ll", " will", phrase)
    phrase = re.sub(r"\'t", " not", phrase)
    phrase = re.sub(r"\'ve", " have", phrase)
    phrase = re.sub(r"\'m", " am", phrase)
    return phrase

In [0]:

stopwords= set(['br', 'the', 'i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've",\
            "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', \
            'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their',\
            'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', \
            'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', \
            'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', \
            'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after',\
            'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further',\
            'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more',\
            'most', 'other', 'some', 'such', 'only', 'own', 'same', 'so', 'than', 'too', 'very', \
            's', 't', 'can', 'will', 'just', 'don', "don't", 'should', "should've", 'now', 'd', 'll', 'm', 'o', 're', \
            've', 'y', 'ain', 'aren', "aren't", 'couldn', "couldn't", 'didn', "didn't", 'doesn', "doesn't", 'hadn',\
            "hadn't", 'hasn', "hasn't", 'haven', "haven't", 'isn', "isn't", 'ma', 'mightn', "mightn't", 'mustn',\
            "mustn't", 'needn', "needn't", 'shan', "shan't", 'shouldn', "shouldn't", 'wasn', "wasn't", 'weren', "weren't", \
            'won', "won't", 'wouldn', "wouldn't"])

In [0]:
from tqdm import tqdm
preprocessed_synopsis = []
# tqdm is for printing the status bar
for sentance in df['Summary'].values:
    sentance = re.sub(r"http\S+", "", sentance)
    sentance = BeautifulSoup(sentance, 'lxml').get_text()
    sentance = decontracted(sentance)
    sentance = re.sub("\S*\d\S*", "", sentance).strip()
    sentance = re.sub('[^A-Za-z]+', ' ', sentance)
    # https://gist.github.com/sebleier/554280
    sentance = ' '.join(e.lower() for e in sentance.split() if e.lower() not in stopwords)
    preprocessed_synopsis.append(sentance.strip())

In [0]:
df['preprocessed_plots']=preprocessed_synopsis

In [0]:
df.columns

Index(['Unnamed: 0', 'Name', 'Genres', 'Summary', 'preprocessed_plots'], dtype='object')

In [0]:

def remove_spaces(x):
    
    return (",").join(x)

In [0]:

df['Genres']=df['Genres'].apply(remove_spaces)

In [0]:
train,test = train_test_split(df, test_size=0.2, random_state = 42)

In [0]:
vectorizer = CountVectorizer(tokenizer = lambda x: x.split(","), binary='true')
y_train = vectorizer.fit_transform(train['Genres']).toarray()
y_test=vectorizer.transform(test['Genres']).toarray()

In [0]:
max(df['Summary'].apply(gdd))

1391

In [0]:
vect=Tokenizer()
vect.fit_on_texts(train['Summary'])
vocab_size = len(vect.word_index) + 1

In [0]:
vocab_size

291212

In [0]:

encoded_docs_train = vect.texts_to_sequences(train['preprocessed_plots'])
max_length = vocab_size
padded_docs_train = sequence.pad_sequences(encoded_docs_train, maxlen=1200, padding='post')
print(padded_docs_train)

[[   835   3888   8389 ...      0      0      0]
 [    59    176    174 ...      0      0      0]
 [105888   4958   2140 ...      0      0      0]
 ...
 [    92   2747  56004 ...      0      0      0]
 [  1118   3993     96 ...      0      0      0]
 [  6349    126   8384 ...      0      0      0]]


In [0]:
encoded_docs_test = vect.texts_to_sequences(test['preprocessed_plots'])
padded_docs_test = sequence.pad_sequences(encoded_docs_test, maxlen=1200, padding='post')

In [0]:
def f1micro(y_true, y_pred):
    return tf.py_func(f1_score(y_true, y_pred,average='mirco'),tf.double)

In [0]:
from keras.layers import Dense, Flatten, LSTM, Conv1D, Conv2D, MaxPooling1D, Dropout, Activation,GlobalMaxPool1D
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.optimizers import Adam
from time import time
from keras.callbacks import TensorBoard
from keras.layers import Dense, Activation, Embedding, Flatten, GlobalMaxPool1D, Dropout, Conv1D
from keras.callbacks import ReduceLROnPlateau, EarlyStopping, ModelCheckpoint
from keras.losses import binary_crossentropy
from keras.optimizers import Adam

from keras.layers.embeddings import Embedding
from keras.callbacks import ReduceLROnPlateau

In [0]:
tensorboard = TensorBoard(log_dir='logs/{}'.format(time()))
filepath="weights.best.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_f1micro', verbose=0, save_best_only=True, mode='max')

In [0]:
checkpoint = ModelCheckpoint(filepath, monitor='val_f1micro', verbose=0, save_best_only=True, mode='max')
model = Sequential()
# model.add(Embedding(max_words, 20, input_length=maxlen))
model.add(Embedding(vocab_size, output_dim=50, input_length=1200))
model.add(Dropout(0.15))
model.add(GlobalMaxPool1D())
model.add(Dense(140, activation='sigmoid'))

model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 1200, 50)          13006850  
_________________________________________________________________
dropout_2 (Dropout)          (None, 1200, 50)          0         
_________________________________________________________________
global_max_pooling1d_2 (Glob (None, 50)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 140)               7140      
Total params: 13,013,990
Trainable params: 13,013,990
Non-trainable params: 0
_________________________________________________________________


In [0]:
model.compile(loss='binary_crossentropy', optimizer='adam')
callbacks = [
    ReduceLROnPlateau(),
    EarlyStopping(patience=4),
    ModelCheckpoint(filepath='model-simple1.hdf5', save_best_only=True)
]

history = model.fit(padded_docs_train, y_train,
                    class_weight='balanced',
                    epochs=3,
                    batch_size=32,
                    validation_split=0.1,
                    callbacks=callbacks)

/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/indexed_slices.py:434: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Train on 55518 samples, validate on 6169 samples
Epoch 1/3
55518/55518 [==============================] - 194s 3ms/step - loss: 0.1023 - val_loss: 0.0627
Epoch 2/3
55518/55518 [==============================] - 193s 3ms/step - loss: 0.0560 - val_loss: 0.0540
Epoch 3/3
55518/55518 [==============================] - 181s 3ms/step - loss: 0.0502 - val_loss: 0.0496


In [0]:
predictions=model.predict([padded_docs_test])
thresholds=[0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9]

In [0]:
for val in thresholds:
    pred=predictions.copy()
  
    pred[pred>=val]=1
    pred[pred<val]=0
  
    precision = precision_score(y_test, pred, average='micro')
    recall = recall_score(y_test, pred, average='micro')
    f1 = f1_score(y_test, pred, average='micro')
   
    print("Micro-average quality numbers")
    print("Precision: {:.4f}, Recall: {:.4f}, F1-measure: {:.4f}".format(precision, recall, f1))

Micro-average quality numbers
Precision: 0.2892, Recall: 0.6319, F1-measure: 0.3968
Micro-average quality numbers
Precision: 0.5081, Recall: 0.4834, F1-measure: 0.4955
Micro-average quality numbers
Precision: 0.6509, Recall: 0.4071, F1-measure: 0.5009
Micro-average quality numbers
Precision: 0.7494, Recall: 0.3522, F1-measure: 0.4792
Micro-average quality numbers
Precision: 0.8101, Recall: 0.3052, F1-measure: 0.4433
Micro-average quality numbers
Precision: 0.8556, Recall: 0.2616, F1-measure: 0.4007
Micro-average quality numbers
Precision: 0.8919, Recall: 0.2151, F1-measure: 0.3466
Micro-average quality numbers
Precision: 0.9224, Recall: 0.1634, F1-measure: 0.2777
Micro-average quality numbers
Precision: 0.9564, Recall: 0.0956, F1-measure: 0.1739


In [0]:
checkpoint = ModelCheckpoint(filepath, monitor='val_f1micro', verbose=0, save_best_only=True, mode='max')
model = Sequential()
model.add(Embedding(vocab_size, 71, input_length=1200))
model.add(Dropout(0.5))
model.add(Conv1D(300, 3, activation='relu'))
model.add(GlobalMaxPool1D())
# model.add(Conv1D(200, 3, activation='relu'))
# model.add(GlobalMaxPool1D())
model.add(Dense(140))
model.add(Activation('sigmoid'))

model.compile(optimizer='adam', loss='binary_crossentropy')
model.summary()
model.fit(padded_docs_train, y_train,
          class_weight='balanced',
          epochs=3,
          batch_size=32,
          validation_split=0.2)
predictions=model.predict(padded_docs_test)

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_5 (Embedding)      (None, 1200, 71)          18469727  
_________________________________________________________________
dropout_5 (Dropout)          (None, 1200, 71)          0         
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 1198, 300)         64200     
_________________________________________________________________
global_max_pooling1d_5 (Glob (None, 300)               0         
_________________________________________________________________
dense_5 (Dense)              (None, 140)               42140     
_________________________________________________________________
activation_3 (Activation)    (None, 140)               0         
Total params: 18,576,067
Trainable params: 18,576,067
Non-trainable params: 0
__________________________________________

/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/indexed_slices.py:434: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Train on 49349 samples, validate on 12338 samples
Epoch 1/3
49349/49349 [==============================] - 542s 11ms/step - loss: 0.0611 - val_loss: 0.0537
Epoch 2/3
49349/49349 [==============================] - 537s 11ms/step - loss: 0.0422 - val_loss: 0.0491
Epoch 3/3
49349/49349 [==============================] - 530s 11ms/step - loss: 0.0363 - val_loss: 0.0506


In [0]:
predictions=model.predict([padded_docs_test])
for val in thresholds:
    print("For threshold: ", val)
    pred=predictions.copy()
  
    pred[pred>=val]=1
    pred[pred<val]=0
  
    precision = precision_score(y_test, pred, average='micro')
    recall = recall_score(y_test, pred, average='micro')
    f1 = f1_score(y_test, pred, average='micro')
   
    print("Micro-average quality numbers")
    print("Precision: {:.4f}, Recall: {:.4f}, F1-measure: {:.4f}".format(precision, recall, f1))

For threshold:  0.1
Micro-average quality numbers
Precision: 0.1734, Recall: 0.8858, F1-measure: 0.2901
For threshold:  0.2
Micro-average quality numbers
Precision: 0.2880, Recall: 0.8046, F1-measure: 0.4241
For threshold:  0.3
Micro-average quality numbers
Precision: 0.3913, Recall: 0.7416, F1-measure: 0.5123
For threshold:  0.4
Micro-average quality numbers
Precision: 0.4862, Recall: 0.6822, F1-measure: 0.5678
For threshold:  0.5
Micro-average quality numbers
Precision: 0.5779, Recall: 0.6276, F1-measure: 0.6017
For threshold:  0.6
Micro-average quality numbers
Precision: 0.6668, Recall: 0.5755, F1-measure: 0.6178
For threshold:  0.7
Micro-average quality numbers
Precision: 0.7527, Recall: 0.5178, F1-measure: 0.6136
For threshold:  0.8
Micro-average quality numbers
Precision: 0.8359, Recall: 0.4507, F1-measure: 0.5856
For threshold:  0.9
Micro-average quality numbers
Precision: 0.9121, Recall: 0.3577, F1-measure: 0.5139


In [0]:
n_classes=140

In [0]:
checkpoint = ModelCheckpoint(filepath, monitor='val_f1micro', verbose=0, save_best_only=True, mode='max')
model = Sequential()
# Configuring the parameters
# model.add(LSTM(6119, input_shape=(timesteps, input_dim)))
model.add(Embedding(vocab_size, output_dim=50, input_length=1200))
model.add(LSTM(128))
# Adding a dropout layer
model.add(Dropout(0.5))
# Adding a dense output layer with sigmoid activation
model.add(Dense(n_classes, activation='sigmoid'))
model.summary()

Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_6 (Embedding)      (None, 1200, 50)          13006850  
_________________________________________________________________
lstm_1 (LSTM)                (None, 128)               91648     
_________________________________________________________________
dropout_6 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_6 (Dense)              (None, 140)               18060     
Total params: 13,116,558
Trainable params: 13,116,558
Non-trainable params: 0
_________________________________________________________________


In [0]:
model.compile(optimizer='adam', loss='binary_crossentropy')
model.summary()

# callbacks = [
#     ReduceLROnPlateau(),
#     EarlyStopping(patience=4),
#     ModelCheckpoint(filepath='model-2.h5', save_best_only=True)
# ]
history = model.fit(padded_docs_train, y_train,
                    class_weight='balanced',
                    epochs=3,
                    batch_size=32,
                    validation_split=0.1,
                    callbacks=[])

Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_6 (Embedding)      (None, 1200, 50)          13006850  
_________________________________________________________________
lstm_1 (LSTM)                (None, 128)               91648     
_________________________________________________________________
dropout_6 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_6 (Dense)              (None, 140)               18060     
Total params: 13,116,558
Trainable params: 13,116,558
Non-trainable params: 0
_________________________________________________________________


/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/indexed_slices.py:434: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Train on 55518 samples, validate on 6169 samples
Epoch 1/3
55518/55518 [==============================] - 2290s 41ms/step - loss: 0.0728 - val_loss: 0.0636
Epoch 2/3
55518/55518 [==============================] - 2330s 42ms/step - loss: 0.0649 - val_loss: 0.0637
Epoch 3/3
55518/55518 [==============================] - 2349s 42ms/step - loss: 0.0646 - val_loss: 0.0635


In [0]:
predictions=model.predict([padded_docs_test])
for val in thresholds:
    print("For threshold: ", val)
    pred=predictions.copy()
  
    pred[pred>=val]=1
    pred[pred<val]=0
  
    precision = precision_score(y_test, pred, average='micro')
    recall = recall_score(y_test, pred, average='micro')
    f1 = f1_score(y_test, pred, average='micro')
   
    print("Micro-average quality numbers")
    print("Precision: {:.4f}, Recall: {:.4f}, F1-measure: {:.4f}".format(precision, recall, f1))

For threshold:  0.1
Micro-average quality numbers
Precision: 0.2996, Recall: 0.3774, F1-measure: 0.3340
For threshold:  0.2
Micro-average quality numbers
Precision: 0.2996, Recall: 0.3774, F1-measure: 0.3340
For threshold:  0.3
Micro-average quality numbers
Precision: 0.3423, Recall: 0.2875, F1-measure: 0.3125
For threshold:  0.4


/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Micro-average quality numbers
Precision: 0.0000, Recall: 0.0000, F1-measure: 0.0000
For threshold:  0.5
Micro-average quality numbers
Precision: 0.0000, Recall: 0.0000, F1-measure: 0.0000
For threshold:  0.6
Micro-average quality numbers
Precision: 0.0000, Recall: 0.0000, F1-measure: 0.0000
For threshold:  0.7
Micro-average quality numbers
Precision: 0.0000, Recall: 0.0000, F1-measure: 0.0000
For threshold:  0.8
Micro-average quality numbers
Precision: 0.0000, Recall: 0.0000, F1-measure: 0.0000
For threshold:  0.9
Micro-average quality numbers
Precision: 0.0000, Recall: 0.0000, F1-measure: 0.0000


In [0]:
checkpoint = ModelCheckpoint(filepath, monitor='val_f1micro', verbose=0, save_best_only=True, mode='max')
model = Sequential()
# Configuring the parameters
# model.add(LSTM(6119, input_shape=(timesteps, input_dim)))
model.add(Embedding(vocab_size, output_dim=50, input_length=1200))
# model.add(LSTM(128))
model.add(LSTM(128, return_sequences=True))  
# Adding a dropout layer
model.add(Dropout(0.5))
model.add(LSTM(64))
model.add(Dropout(0.5))
# Adding a dense output layer with sigmoid activation
model.add(Dense(n_classes, activation='sigmoid'))
model.summary()
model.compile(optimizer='adam', loss='binary_crossentropy')

history = model.fit(padded_docs_train, y_train,
                    class_weight='balanced',
                    epochs=3,
                    batch_size=32,
                    validation_split=0.1,
                    callbacks=[])

Model: "sequential_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_7 (Embedding)      (None, 1200, 50)          13006850  
_________________________________________________________________
lstm_2 (LSTM)                (None, 1200, 128)         91648     
_________________________________________________________________
dropout_7 (Dropout)          (None, 1200, 128)         0         
_________________________________________________________________
lstm_3 (LSTM)                (None, 64)                49408     
_________________________________________________________________
dropout_8 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_7 (Dense)              (None, 140)               9100      
Total params: 13,157,006
Trainable params: 13,157,006
Non-trainable params: 0
__________________________________________

/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/indexed_slices.py:434: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Train on 55518 samples, validate on 6169 samples
Epoch 1/3
55518/55518 [==============================] - 3691s 66ms/step - loss: 0.0763 - val_loss: 0.0637
Epoch 2/3
55518/55518 [==============================] - 3631s 65ms/step - loss: 0.0655 - val_loss: 0.0637
Epoch 3/3
55518/55518 [==============================] - 3598s 65ms/step - loss: 0.0652 - val_loss: 0.0636


In [0]:
predictions=model.predict([padded_docs_test])
for val in thresholds:
    print("For threshold: ", val)
    pred=predictions.copy()
  
    pred[pred>=val]=1
    pred[pred<val]=0
  
    precision = precision_score(y_test, pred, average='micro')
    recall = recall_score(y_test, pred, average='micro')
    f1 = f1_score(y_test, pred, average='micro')
   
    print("Micro-average quality numbers")
    print("Precision: {:.4f}, Recall: {:.4f}, F1-measure: {:.4f}".format(precision, recall, f1))

For threshold:  0.1
Micro-average quality numbers
Precision: 0.2996, Recall: 0.3774, F1-measure: 0.3340
For threshold:  0.2
Micro-average quality numbers
Precision: 0.2996, Recall: 0.3774, F1-measure: 0.3340
For threshold:  0.3
Micro-average quality numbers
Precision: 0.3423, Recall: 0.2875, F1-measure: 0.3125
For threshold:  0.4


/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Micro-average quality numbers
Precision: 0.0000, Recall: 0.0000, F1-measure: 0.0000
For threshold:  0.5
Micro-average quality numbers
Precision: 0.0000, Recall: 0.0000, F1-measure: 0.0000
For threshold:  0.6
Micro-average quality numbers
Precision: 0.0000, Recall: 0.0000, F1-measure: 0.0000
For threshold:  0.7
Micro-average quality numbers
Precision: 0.0000, Recall: 0.0000, F1-measure: 0.0000
For threshold:  0.8
Micro-average quality numbers
Precision: 0.0000, Recall: 0.0000, F1-measure: 0.0000
For threshold:  0.9
Micro-average quality numbers
Precision: 0.0000, Recall: 0.0000, F1-measure: 0.0000


In [0]:
val=0.2
pred=predictions.copy()
pred[pred>=val]=1
pred[pred<val]=0


In [0]:
vectorizer.inverse_transform(pred[1000]),vectorizer.inverse_transform(y_test[5])

([array(['children’s books', 'fiction', 'nonfiction'], dtype='<U51')],
 [array(['cooking', 'cooking methods', 'nonfiction'], dtype='<U51')])

array([3934,  222,  406, ...,    0,    0,    0], dtype=int32)

In [0]:
model = Sequential()
model.add(Embedding(vocab_size, output_dim=50, input_length=1200))
# model.add(layers.Embedding(vocab_size, embedding_dim,  
#                            input_length=1200, 
#                            trainable=True))
model.add(layers.GlobalMaxPool1D())
model.add(layers.Dense(100, activation='sigmoid'))
# classifier.add(Dropout(.70))

model.add(layers.Dense(n_classes, activation='sigmoid'))
model.compile(optimizer='adam',
              loss='binary_crossentropy')
model.summary()

# model.compile(optimizer='adam', loss='binary_crossentropy')

model.compile(loss='binary_crossentropy', 
                  optimizer='adam')

model.fit(padded_docs_train, y_train,
                        epochs=2,
                        verbose=False,
                        validation_data=(padded_docs_test, y_test),
                        batch_size=16)


Model: "sequential_8"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_8 (Embedding)      (None, 1200, 50)          13006850  
_________________________________________________________________
global_max_pooling1d_6 (Glob (None, 50)                0         
_________________________________________________________________
dense_8 (Dense)              (None, 100)               5100      
_________________________________________________________________
dense_9 (Dense)              (None, 140)               14140     
Total params: 13,026,090
Trainable params: 13,026,090
Non-trainable params: 0
_________________________________________________________________


/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/indexed_slices.py:434: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


In [0]:
predictions=model.predict([padded_docs_test])
for val in thresholds:
    print("For threshold: ", val)
    pred=predictions.copy()
  
    pred[pred>=val]=1
    pred[pred<val]=0
  
    precision = precision_score(y_test, pred, average='micro')
    recall = recall_score(y_test, pred, average='micro')
    f1 = f1_score(y_test, pred, average='micro')
   
    print("Micro-average quality numbers")
    print("Precision: {:.4f}, Recall: {:.4f}, F1-measure: {:.4f}".format(precision, recall, f1))

For threshold:  0.1
Micro-average quality numbers
Precision: 0.3682, Recall: 0.6407, F1-measure: 0.4676
For threshold:  0.2
Micro-average quality numbers
Precision: 0.5765, Recall: 0.5255, F1-measure: 0.5498
For threshold:  0.3
Micro-average quality numbers
Precision: 0.7045, Recall: 0.4635, F1-measure: 0.5591
For threshold:  0.4
Micro-average quality numbers
Precision: 0.7866, Recall: 0.4181, F1-measure: 0.5459
For threshold:  0.5
Micro-average quality numbers
Precision: 0.8339, Recall: 0.3780, F1-measure: 0.5202
For threshold:  0.6
Micro-average quality numbers
Precision: 0.8712, Recall: 0.3378, F1-measure: 0.4868
For threshold:  0.7
Micro-average quality numbers
Precision: 0.9020, Recall: 0.2965, F1-measure: 0.4463
For threshold:  0.8
Micro-average quality numbers
Precision: 0.9296, Recall: 0.2505, F1-measure: 0.3946
For threshold:  0.9
Micro-average quality numbers
Precision: 0.9597, Recall: 0.1814, F1-measure: 0.3051


In [0]:
model = Sequential()
model.add(Embedding(vocab_size, output_dim=50, input_length=1200))
# model.add(layers.Embedding(vocab_size, embedding_dim,  
#                            input_length=1200, 
#                            trainable=True))
model.add(layers.GlobalMaxPool1D())
model.add(layers.Dense(100, activation='sigmoid'))
model.add(Dropout(.70))
model.add(layers.Dense(100, activation='sigmoid'))
model.add(layers.Dense(n_classes, activation='sigmoid'))
model.compile(optimizer='adam',
              loss='binary_crossentropy')
model.summary()
model.fit(padded_docs_train, y_train,
                        epochs=5,
                        verbose=False,
                        validation_data=(padded_docs_test, y_test),
                        batch_size=16)

Model: "sequential_9"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_9 (Embedding)      (None, 1200, 50)          13006850  
_________________________________________________________________
global_max_pooling1d_7 (Glob (None, 50)                0         
_________________________________________________________________
dense_10 (Dense)             (None, 100)               5100      
_________________________________________________________________
dropout_9 (Dropout)          (None, 100)               0         
_________________________________________________________________
dense_11 (Dense)             (None, 100)               10100     
_________________________________________________________________
dense_12 (Dense)             (None, 140)               14140     
Total params: 13,036,190
Trainable params: 13,036,190
Non-trainable params: 0
__________________________________________

/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/indexed_slices.py:434: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


In [0]:
predictions=model.predict([padded_docs_test])
for val in thresholds:
    print("For threshold: ", val)
    pred=predictions.copy()
  
    pred[pred>=val]=1
    pred[pred<val]=0
  
    precision = precision_score(y_test, pred, average='micro')
    recall = recall_score(y_test, pred, average='micro')
    f1 = f1_score(y_test, pred, average='micro')
   
    print("Micro-average quality numbers")
    print("Precision: {:.4f}, Recall: {:.4f}, F1-measure: {:.4f}".format(precision, recall, f1))

For threshold:  0.1
Micro-average quality numbers
Precision: 0.3599, Recall: 0.6773, F1-measure: 0.4700
For threshold:  0.2
Micro-average quality numbers
Precision: 0.5629, Recall: 0.5613, F1-measure: 0.5621
For threshold:  0.3
Micro-average quality numbers
Precision: 0.6958, Recall: 0.4912, F1-measure: 0.5759
For threshold:  0.4
Micro-average quality numbers
Precision: 0.7823, Recall: 0.4391, F1-measure: 0.5625
For threshold:  0.5
Micro-average quality numbers
Precision: 0.8384, Recall: 0.3962, F1-measure: 0.5381
For threshold:  0.6
Micro-average quality numbers
Precision: 0.8766, Recall: 0.3596, F1-measure: 0.5100
For threshold:  0.7
Micro-average quality numbers
Precision: 0.9079, Recall: 0.3228, F1-measure: 0.4763
For threshold:  0.8
Micro-average quality numbers
Precision: 0.9282, Recall: 0.2897, F1-measure: 0.4416
For threshold:  0.9
Micro-average quality numbers
Precision: 0.9526, Recall: 0.2332, F1-measure: 0.3747


In [0]:
n_classes=140

In [0]:
model = Sequential()
# model.add(Conv1D(180, 3, input_shape=(timesteps, input_dim), activation='relu'))
model.add(Embedding(vocab_size, 71, input_length=1200))
model.add(Conv1D(64, 3, activation='sigmoid'))
# model.add(Dropout(0.70))
model.add(Conv1D(100, 3, activation='sigmoid'))
model.add(Conv1D(100, 3, activation='sigmoid'))
model.add(Conv1D(48, 3, activation='sigmoid'))
model.add(Flatten())
# model.add(Dense(6, activation='softmax'))
model.add(Dense(n_classes))
# model.add(Dropout(0.70))
model.compile(optimizer='adam', loss='binary_crossentropy')
# model.compile(loss='categorical_crossentropy',
#               optimizer='adam',
#               metrics=['accuracy'])
# model.compile(loss="binary_crossentropy", optimizer="adam", metrics=['accuracy']) 
model.summary()
history=model.fit(padded_docs_train, y_train,
                        epochs=5,
                        verbose=False,
                        validation_split=0.1,
                        batch_size=16,
                  callbacks=[])

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 1200, 71)          20676052  
_________________________________________________________________
conv1d_5 (Conv1D)            (None, 1198, 64)          13696     
_________________________________________________________________
conv1d_6 (Conv1D)            (None, 1196, 100)         19300     
_________________________________________________________________
conv1d_7 (Conv1D)            (None, 1194, 100)         30100     
_________________________________________________________________
conv1d_8 (Conv1D)            (None, 1192, 48)          14448     
_________________________________________________________________
flatten_2 (Flatten)          (None, 57216)             0         
_________________________________________________________________
dense_1 (Dense)              (None, 140)              

/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/indexed_slices.py:434: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


In [0]:
predictions=model.predict([padded_docs_test])
for val in thresholds:
    print("For threshold: ", val)
    pred=predictions.copy()
  
    pred[pred>=val]=1
    pred[pred<val]=0
  
    precision = precision_score(y_test, pred, average='micro')
    recall = recall_score(y_test, pred, average='micro')
    f1 = f1_score(y_test, pred, average='micro')
   
    print("Micro-average quality numbers")
    print("Precision: {:.4f}, Recall: {:.4f}, F1-measure: {:.4f}".format(precision, recall, f1))

For threshold:  0.1
Micro-average quality numbers
Precision: 0.0100, Recall: 0.0377, F1-measure: 0.0158
For threshold:  0.2
Micro-average quality numbers
Precision: 0.0100, Recall: 0.0377, F1-measure: 0.0158
For threshold:  0.3
Micro-average quality numbers
Precision: 0.0100, Recall: 0.0377, F1-measure: 0.0158
For threshold:  0.4
Micro-average quality numbers
Precision: 0.0100, Recall: 0.0377, F1-measure: 0.0158
For threshold:  0.5
Micro-average quality numbers
Precision: 0.0100, Recall: 0.0377, F1-measure: 0.0158
For threshold:  0.6
Micro-average quality numbers
Precision: 0.0100, Recall: 0.0377, F1-measure: 0.0158
For threshold:  0.7
Micro-average quality numbers
Precision: 0.0100, Recall: 0.0377, F1-measure: 0.0158
For threshold:  0.8
Micro-average quality numbers
Precision: 0.0100, Recall: 0.0377, F1-measure: 0.0158
For threshold:  0.9
Micro-average quality numbers
Precision: 0.0100, Recall: 0.0377, F1-measure: 0.0158


In [0]:
model = Sequential()
# model.add(Conv1D(180, 3, input_shape=(timesteps, input_dim), activation='relu'))
model.add(Embedding(vocab_size, 71, input_length=1200))
model.add(Conv1D(64, 3, activation='sigmoid'))
model.add(Dropout(0.70))
model.add(Conv1D(100, 3, activation='sigmoid'))
model.add(Dropout(0.70))
model.add(Conv1D(100, 3, activation='sigmoid'))
model.add(Dropout(0.70))
model.add(Conv1D(48, 3, activation='sigmoid'))
model.add(Flatten())
# model.add(Dense(6, activation='softmax'))
model.add(Dense(n_classes))

model.compile(optimizer='adam', loss='binary_crossentropy')
# model.compile(loss='categorical_crossentropy',
#               optimizer='adam',
#               metrics=['accuracy'])
# model.compile(loss="binary_crossentropy", optimizer="adam", metrics=['accuracy']) 
model.summary()
model.fit(padded_docs_train, y_train,
                        epochs=10,
                        verbose=False,
                        validation_split=0.1,
                        batch_size=16)

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_5 (Embedding)      (None, 1200, 71)          20676052  
_________________________________________________________________
conv1d_17 (Conv1D)           (None, 1198, 64)          13696     
_________________________________________________________________
dropout_7 (Dropout)          (None, 1198, 64)          0         
_________________________________________________________________
conv1d_18 (Conv1D)           (None, 1196, 100)         19300     
_________________________________________________________________
dropout_8 (Dropout)          (None, 1196, 100)         0         
_________________________________________________________________
conv1d_19 (Conv1D)           (None, 1194, 100)         30100     
_________________________________________________________________
dropout_9 (Dropout)          (None, 1194, 100)        

/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/indexed_slices.py:434: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


In [0]:
predictions=model.predict([padded_docs_test])
for val in thresholds:
    print("For threshold: ", val)
    pred=predictions.copy()
  
    pred[pred>=val]=1
    pred[pred<val]=0
  
    precision = precision_score(y_test, pred, average='micro')
    recall = recall_score(y_test, pred, average='micro')
    f1 = f1_score(y_test, pred, average='micro')
   
    print("Micro-average quality numbers")
    print("Precision: {:.4f}, Recall: {:.4f}, F1-measure: {:.4f}".format(precision, recall, f1))

For threshold:  0.1
Micro-average quality numbers
Precision: 0.0875, Recall: 0.1838, F1-measure: 0.1186
For threshold:  0.2
Micro-average quality numbers
Precision: 0.0875, Recall: 0.1838, F1-measure: 0.1186
For threshold:  0.3
Micro-average quality numbers
Precision: 0.0875, Recall: 0.1838, F1-measure: 0.1186
For threshold:  0.4
Micro-average quality numbers
Precision: 0.0875, Recall: 0.1838, F1-measure: 0.1186
For threshold:  0.5
Micro-average quality numbers
Precision: 0.0875, Recall: 0.1838, F1-measure: 0.1186
For threshold:  0.6
Micro-average quality numbers
Precision: 0.0875, Recall: 0.1838, F1-measure: 0.1186
For threshold:  0.7
Micro-average quality numbers
Precision: 0.0875, Recall: 0.1838, F1-measure: 0.1186
For threshold:  0.8
Micro-average quality numbers
Precision: 0.0875, Recall: 0.1838, F1-measure: 0.1186
For threshold:  0.9
Micro-average quality numbers
Precision: 0.0875, Recall: 0.1838, F1-measure: 0.1186
